In [1]:
import pandas as pd
import os
import sys
import hdbscan
import pickle
print(sys.version)
import numpy as np

3.6.3 (default, Oct 13 2017, 12:12:21) 
[GCC Intel(R) C++ gcc 4.8.5 mode]


In [2]:
pd.__version__

'0.25.1'

In [3]:
filename = "../gd1.h5"

In [4]:
dataset = pd.read_hdf(filename, "df")

In [5]:
dataset.columns

Index(['source_id_x', 'ra', 'dec', 'parallax_x', 'parallax_error_x', 'pmra_x',
       'pmra_error_x', 'pmdec_x', 'pmdec_error_x', 'ra_parallax_corr',
       'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr',
       'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr',
       'pmra_pmdec_corr', 'visibility_periods_used', 'g',
       'phot_g_mean_flux_over_error', 'bp', 'phot_bp_mean_flux_over_error',
       'rp', 'phot_rp_mean_flux_over_error', 'phot_bp_rp_excess_factor',
       'astrometric_chi2_al_x', 'astrometric_n_good_obs_al_x', 'pm_mask',
       'gi_cmd_mask', 'phi1', 'phi2', 'pm_phi1_cosphi2_no_reflex',
       'pm_phi2_no_reflex', 'pm_phi1_cosphi2', 'pm_phi2', 'stream_track_mask',
       'g_bp', 'g_rp', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'wx', 'wy', 'wz',
       'wrad', 'wtheta', 'wphi', 'bp_rp', 'bp_g', 'phot_g_mean_mag',
       'parallax_y', 'parallax_error_y', 'astrometric_chi2_al_y',
       'astrometric_n_good_obs_al_y', 'source_id_y', 'pmra_y', 'pm

In [6]:
dataset["label"] = dataset.pm_mask & dataset.gi_cmd_mask & dataset.stream_track_mask

In [54]:
datasets = {}
datasets["non_gd1"] = dataset[~dataset["label"]]
datasets["gd1"] = dataset[dataset["label"]]

In [85]:
sample_size = 1*10**5
sample = pd.concat((datasets["gd1"], datasets["non_gd1"].sample(sample_size) ), axis=0, ignore_index=True)

In [86]:
sample_clust = sample[["ra", "dec", "pmdec_x", "pmra_x"]]

In [124]:
clusterer = hdbscan.HDBSCAN(min_cluster_size = 1000, metric="mahalanobis", allow_single_cluster=True, V = np.cov(sample_clust.T))

In [125]:
clusterer.fit(sample_clust)
pickle.dump( clusterer, open( "cluster_test.p", "wb" ) )



In [97]:
test = pickle.load(open("cluster_test.p", "rb"))

In [98]:
test

HDBSCAN(algorithm='best', allow_single_cluster=True, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_method='eom',
        core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='mahalanobis', min_cluster_size=500, min_samples=None, p=None,
        prediction_data=False)

In [120]:
test.labels_
n_clusters = len(np.unique(test.labels_)) - 1
predicted_labels = (test.labels_ + 1) == 2

In [121]:
from sklearn import metrics

In [122]:
precision = metrics.precision_score(sample["label"], predicted_labels)
recall = metrics.recall_score(sample["label"], predicted_labels)
f1 = metrics.f1_score(sample["label"], predicted_labels)

In [123]:
print(precision, recall, f1)

0.9278996865203761 0.2991409802930773 0.45242644249140235


In [108]:
n_clusters

2

In [109]:
df = pd.DataFrame({"label": test.labels_})

In [115]:
df.groupby("label").count()

""
label
-1
0
1


In [117]:
from collections import Counter

In [118]:
c = Counter(test.labels_)

In [119]:
c

Counter({-1: 27487, 0: 73854, 1: 638})